In [1]:
import json
import requests

In [2]:
from extract_features_entities_rasa import RasaExtractor

ex = RasaExtractor()

In [3]:
#cargar Rasa + duckling
RASA_HOST = '127.0.0.1'
RASA_PORT = 5005

rasa_endpoint = "http://{0}:{1}/model/parse".format(RASA_HOST, RASA_PORT)

In [4]:
#cargo los claims
with open('data/claims.json') as json_file: 
        claims = json.load(json_file) 

with open('data/output_all.jsonl', 'r') as json_file: 
    json_list = list(json_file)

sentences = [json.loads(sentence) for sentence in json_list]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [6]:
#tokenizer = AutoTokenizer.from_pretrained("mrm8488/RuPERTa-base-finetuned-pawsx-es")
#model = AutoModelForSequenceClassification.from_pretrained("mrm8488/RuPERTa-base-finetuned-pawsx-es")

In [7]:
tokenizer_b = AutoTokenizer.from_pretrained("mrm8488/electricidad-base-finetuned-pawsx-es")
model_b = AutoModelForSequenceClassification.from_pretrained("mrm8488/electricidad-base-finetuned-pawsx-es")

In [8]:
for claim in claims:
    claim = claim.get("claim")
    cl_ents = ex.get_claim_entities(claim)
    for sentence in sentences:
        sentence = sentence.get("sentence_text")
        se_ents = ex.get_claim_entities(sentence)
        similarity_dicts = ex.output(cl_ents, se_ents)
        if similarity_dicts['matches_points'] >= 1:
            
            claims_pair = tokenizer_b.encode_plus(claim, sentence, return_tensors="pt")
            claims_pair_logits = model_b(**claims_pair)[0]
            claims_pair_results = torch.softmax(claims_pair_logits, dim=1).tolist()[0]
            if claims_pair_results[1] > 0.7:
                import pdb; pdb.set_trace()
                
    print ('...')

...
...
...


KeyError: 'unit'

In [13]:
classes = ["no son lo mismo", "son lo mismo"]

sequence_0 = "La inflación en 2016 fue de 50%"
sequence_1 = "En el 2016 tuvimos una inflación cercana al 50%"
sequence_2 = "La inflación es un problema que Argentina no puede solucionar"

paraphrase = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")
not_paraphrase = tokenizer.encode_plus(sequence_0, sequence_2, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase)[0]
not_paraphrase_classification_logits = model(**not_paraphrase)[0]

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

print("Should be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results[i] * 100)}%")

print("\nShould not be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(not_paraphrase_results[i] * 100)}%")

Should be paraphrase
no son lo mismo: 19%
son lo mismo: 81%

Should not be paraphrase
no son lo mismo: 66%
son lo mismo: 34%


In [21]:
claims_pair = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")
claims_pair_logits = model(**claims_pair)[0]
claims_pair_results = torch.softmax(claims_pair_logits, dim=1).tolist()[0]

In [15]:
for i in range(len(classes)):
    print(f"{classes[i]}: {round(claims_pair_results[i] * 100)}%")

no son lo mismo: 19%
son lo mismo: 81%


In [22]:
claims_pair_results

[0.18940554559230804, 0.8105944991111755]

In [40]:
claims = [
    'La inflación en 2016 fue de 50%',
    'En el 2016 tuvimos una inflación cercana al 50%',
    'La inflación es un problema que Argentina no puede solucionar',
    'En los 12 últimos meses la inflación fue la más baja de los últimos siete años.',
    'Hay que trabajar medio año para el Estado.',
    'Trabajamos la mitad del año para pagarle impuestos al Estado',
    'Entre 2003 y 2011 creamos 300 mil empleos.',
    'En la industria generamos más de un millón de puestos de trabajo.',
    'Creamos más de un millón de empleos en la industria',
]

In [27]:
similar_claims = []
for i in range(len(claims)):
    for j in range(len(claims)):
        if i == j:
            continue
        claims_pair = tokenizer.encode_plus(claims[i], claims[j], return_tensors="pt")
        claims_pair_logits = model(**claims_pair)[0]
        claims_pair_results = torch.softmax(claims_pair_logits, dim=1).tolist()[0]
        if claims_pair_results[1] > 0.7:
            similar_claims.append([claims[i], claims[j], claims_pair_results[1]])

In [41]:
similar_claims_b = []
for i in range(len(claims)):
    for j in range(len(claims)):
        if i == j:
            continue
        claims_pair = tokenizer_b.encode_plus(claims[i], claims[j], return_tensors="pt")
        claims_pair_logits = model_b(**claims_pair)[0]
        claims_pair_results = torch.softmax(claims_pair_logits, dim=1).tolist()[0]
        if claims_pair_results[1] > 0.7:
            similar_claims_b.append([claims[i], claims[j], claims_pair_results[1]])

In [28]:
similar_claims

[['La inflación en 2016 fue de 50%',
  'En el 2016 tuvimos una inflación cercana al 50%',
  0.8105944991111755],
 ['En el 2016 tuvimos una inflación cercana al 50%',
  'La inflación en 2016 fue de 50%',
  0.806519627571106],
 ['Trabajamos la mitad del año para pagarle impuestos al Estado',
  'La inflación en 2016 fue de 50%',
  0.7567902207374573],
 ['Trabajamos la mitad del año para pagarle impuestos al Estado',
  'Hay que trabajar medio año para el Estado.',
  0.721567690372467]]

In [42]:
similar_claims_b

[['La inflación en 2016 fue de 50%',
  'En el 2016 tuvimos una inflación cercana al 50%',
  0.805130660533905],
 ['En el 2016 tuvimos una inflación cercana al 50%',
  'La inflación en 2016 fue de 50%',
  0.8785433769226074],
 ['Hay que trabajar medio año para el Estado.',
  'Trabajamos la mitad del año para pagarle impuestos al Estado',
  0.7958112359046936],
 ['En la industria generamos más de un millón de puestos de trabajo.',
  'Creamos más de un millón de empleos en la industria',
  0.9922629594802856],
 ['Creamos más de un millón de empleos en la industria',
  'En la industria generamos más de un millón de puestos de trabajo.',
  0.9160385131835938]]